In [9]:
from data.function import Stock,Categories

In [10]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

PTT = stock("PTT")
stock_price = PTT.get_all_stock_price(start = "2018-01-01",end = "2022-12-31")# Get stock price
df = pd.DataFrame(stock_price)# convert data to pandas dataframe
print(df)
fig = go.Figure(data=[go.Candlestick(x=df[0],# datetime
                open=df[1],# open
                high=df[2],# high
                low=df[3],# low
                close=df[4])])# close

fig.show()

                        0          1          2          3          4  \
0     2018-01-03 00:00:00  44.200001  45.000000  44.000000  45.000000   
1     2018-01-04 00:00:00  45.200001  47.200001  45.000000  47.200001   
2     2018-01-05 00:00:00  47.200001  47.400002  46.400002  46.799999   
3     2018-01-08 00:00:00  46.799999  47.200001  46.599998  46.599998   
4     2018-01-09 00:00:00  46.599998  46.599998  46.599998  46.599998   
...                   ...        ...        ...        ...        ...   
1209  2022-12-26 00:00:00  31.500000  32.250000  31.500000  32.250000   
1210  2022-12-27 00:00:00  32.500000  32.750000  32.250000  32.500000   
1211  2022-12-28 00:00:00  32.500000  32.750000  32.250000  32.750000   
1212  2022-12-29 00:00:00  32.750000  33.000000  32.500000  32.750000   
1213  2022-12-30 00:00:00  33.000000  33.500000  32.750000  33.250000   

              5  
0      81239000  
1     124259000  
2      87860000  
3      63976000  
4       5658000  
...         ...